Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [4]:
NAME = "Nick"
COLLABORATORS = "Nick"

---

In [1]:
import numpy as np
from numpy.testing import assert_allclose

# 1. Степенная итерация

Дана квадратная $n \times n$ матрица $A$. Алгоритм степенной итерации позволяет найти максимальное собственное значение $\lambda$ матрицы $A$. Для начала необходимо задать начальное приближение собственного вектора $x_0$. 


Модифицированный алгоритм степенной итерации:

1. $y_{k+1} = A x_k$

2. $x_{k+1} = \cfrac{y_{k+1}}{||y_{k+1}||}$

3. $\lambda^{(k)} = (x_k, y_{k+1})$

где $k$ - номер итерации.

Напишите функцию, реализующую поиск максимального собственного значения заданной матрицы с помощью алгоритма степенной итерации.

In [24]:
def power_iter(a, vec, eps=1e-3, maxiter=1000):
    """Реализует алгоритм степенной итерации поиска максимального собственного значения матрицы `a` 
        с начальным приближением собственного вектора vec.
    
    Parameters
    ----------
    a : array-like of floats, shape (n, n)
        Введённая матрица
    vec : array-like of floats, shape (n,)
        Введённое начальное приближение собственного вектора
    eps : float
        Введённая абсолютная точность
    maxiter : integer
        Максимальное количество итераций
    
    Returns
    -------
    lm : float
        Приближение максимального собственного значения
    outvec : array of floats, shape (n,)
        Приближение собственного вектора
    iters : integer
        Количество итераций
    """
    a = np.asarray(a, dtype=float)
    if a.ndim != 2:
        raise ValueError("a.ndim = %s, expected 2" % a.ndim)
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    if (a.shape[0] != a.shape[1]):
        raise ValueError("a is not square")
        
    if (a.shape[0] != vec.shape[0]):
        raise ValueError("vec should be of length %s" % a.shape[0])
    
    # YOUR CODE HERE
    lm = 0
    y = np.zeros(shape=vec.shape)
    for iter in range(0, maxiter):
      y = a @ vec
      lm = np.dot(vec, y)
      if np.linalg.norm((y - lm * vec)) / np.linalg.norm(vec) < eps:
        return lm, vec, iter
      vec = y / np.linalg.norm(y)
    return lm, vec, maxiter

Протестируйте свою функцию на следующих примерах:

In [25]:
A = np.asarray([[4, 34],[7, 1]])

np.random.seed(123)
x = np.random.random(size=2)

l, e, k = power_iter(A, x)
assert_allclose(A @ e, l * e, atol=1e-3)

l, e, k = power_iter(A, x, 1e-5)
assert_allclose(A @ e, l * e, atol=1e-5)


# 2. Обратная итерация

Этот метод может использоваться, например, для поиска собственных векторов матрицы, если уже имеется достаточно хорошее приближение её максимального собственного значения.

Для реализации алгоритма необходимо задать начальные приближения для собственного вектора $x_0$ и собственного значения $\mu$.

На каждой итерации метода решается система уравнений:
$$(A - \lambda^{(k)} I) y_{k+1} = x_k,$$
приэтом $\lambda^{(0)} = \mu$. После этого за новое приближение собственного вектора принимается нормированный вектор $y_{k+1}$:
$$x_{k+1} = \cfrac{y_{k+1}}{||y_{k+1}||}.$$

In [26]:
def inverse_iter(a, vec, mu, eps=1e-3, maxiter=1000):
    """Реализует алгоритм степенной итерации поиска максимального собственного значения матрицы a 
        с начальным приближением собственного вектора vec и начальным приближением собственного значения mu
    
    Parameters
    ----------
    a : array-like of floats, shape (n, n)
        Введённая матрица
    vec : array-like of floats, shape (n,)
        Введённое начальное приближение собственного вектора
    mu : float
        Введённое начальное приближение собственного значения
    eps : float
        Введённая точность 
    maxiter : integer
        Максимальное количество итераций
    
    Returns
    -------
    lm : float
        Приближение максимального собственного значения
    outvec : array of floats, shape (n,)
        Приближение собственного вектора
    iters : integer
        Количество итераций
    """
    a = np.asarray(a, dtype=float)
    if a.ndim != 2:
        raise ValueError("a.ndim = %s, expected 2" % a.ndim)
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    if (a.shape[0] != a.shape[1]):
        raise ValueError("a is not square")
        
    if (a.shape[0] != vec.shape[0]):
        raise ValueError("vec should be of length %s" % a.shape[0])
    
    # YOUR CODE HERE
    lm = mu
    x = vec.copy()
    iter = 0
    ax = a @ x
    for iter in range(maxiter):
        if np.linalg.norm(ax - lm * x) < eps:
            return lm, x, iter
        try:
            y = np.linalg.solve(a - lm * np.eye(a.shape[0]), x)
        except:
            lm += 1e-3 * np.random.normal()
            y = np.linalg.solve(a - lm * np.eye(a.shape[0]), x)
        x = y / np.linalg.norm(y)
        ax = a @ x
        lm = x @ ax
        iter += 1
    return lm, x, maxiter

Протестируйте свою функцию на следующих примерах:

In [27]:
A = np.asarray([[4, 34],
                [7, 1]])
x = np.random.random(size=2)

l, e, k = inverse_iter(A, x, 1.)
assert_allclose(A @ e, l * e, atol=1e-3)

l, e, k = inverse_iter(A, x, 100.)
assert_allclose(A @ e, l * e, atol=1e-3)


## Пример

Рассмотрите матрицу 
$$ A = 
\begin{bmatrix}
3 & 1 & 0 & 0\\ 
1 & 2 & 0 & 1\\ 
0 & 0 & 1 & 1\\ 
0 & 1 & 1 & 1
\end{bmatrix}.$$

In [28]:
# Определите матрицу `А` и задайте начальное приближение для собственного вектора
# (используйте случайный начальный вектор)

def A_and_x(seed=1234):
    """Возвращает матрицу `А` и случайный начальный вектор."""

    # YOUR CODE HERE
    np.random.seed(seed)
    A = np.array([[3, 1, 0, 0], [1, 2, 0, 1], [0, 0, 1, 1], [0, 1, 1, 1]])
    size = A.shape[0]
    x = np.random.random(size=size)
    return A, x

Найдите максимальное собственное число этой матрицы тремя разными методами.

### 1) Степенная итерация

In [29]:
A, x = A_and_x()

l, e, k = power_iter(A, x)
assert_allclose(A @ e, l * e, atol=1.6e-2)


### 2) Обратная итерация с $\mu = 3.5$

In [30]:
A, x = A_and_x()

l, e, k = inverse_iter(A, x, 3.5)
assert_allclose(A @ e, l * e, atol=1e-3)


### 3) Обратная итерация с $\mu = 3.7$

In [31]:
A, x = A_and_x()

l, e, k = inverse_iter(A, x, 3.7)
assert_allclose(A @ e, l * e, atol=1e-3)


Сколько шагов $k$ требуется в каждом случае для того, чтобы получить настоящий собственный вектор $x^*$ с точностью $||x^* - x_k||_2 < 10^{-3}$?

In [32]:
# YOUR CODE HERE
A, x = A_and_x()

l, e, k = power_iter(A, x)
w, v = np.linalg.eigh(A)
print(np.linalg.norm(v[:,3] + e), k)

l, e, k = inverse_iter(A, x, 3.5)
w, v = np.linalg.eigh(A)
print(np.linalg.norm(v[:,3] + e), k)

l, e, k = inverse_iter(A, x, 3.7)
w, v = np.linalg.eigh(A)
print(np.linalg.norm(v[:,3] + e), k)

0.0005026681865792987 16
2.2517802156321836e-06 3
6.82422150294179e-05 2
